In [1]:
import urllib.request as req
# pip install 
from fastkml import kml
import pandas as pd
import re

In [2]:
# using the map from https://hargitamegye.ro/vadkarok/interaktiv-terkep.html
# downloading in KML format
url = 'http://www.google.com/maps/d/kml?forcekml=1&mid=1sIXAUcYDAkCBUzw_6DZelr1rt48'

In [3]:
fname, _ = req.urlretrieve(url)
k = kml.KML()

with open(fname, 'rt', encoding='utf-8') as f:
    k.from_string(f.read().encode('utf-8'))

In [4]:
columns_raw = ['year', 'location', 'lat', 'lon', 'description', 'media']
df = pd.DataFrame(columns=columns_raw)

In [5]:
folders = next(k.features()).features()

In [6]:
# parse KML, insert data into dataframe
for f in folders:
    for pm in f.features():
        lon, lat, _ =  pm.geometry.coords[0]
        media_ln = ''
        if pm.extended_data is not None:
            media_ln = pm.extended_data.elements[0].value
        df = df.append({
                    'year': f.name, 
                    'location': pm.name, 
                    'lat': lat, 
                    'lon': lon, 
                    'description': pm.description, 
                    'media': media_ln}, 
                ignore_index=True)

In [7]:
def remove_links(desc):
    '''
        Input: description extracted from KML files
        Output: description without the links of attached images
    '''
    desc = desc.lstrip().rstrip()
    desc = re.sub(r'<img[^>]*/>', '', desc)
    desc = re.sub(r'<br>', '', desc)
    return desc

In [8]:
df['description'] = df['description'].apply(remove_links)

In [9]:
dp = re.compile('([0-9]{4})[. ]+(január|február|március|április|május|június|július|' + 
                'augusztus|szeptember|október|november|december|[0-9]{1,2})[. ]+([0-9]{1,2})')

def standardize_date(date_groups):
    '''
        Input: result of regexp match with 3 groups, corresponding to (year, month, day)
        Output: (year, month, day) tuple, where month is converted to numerical format
    '''
    months_hu = [u'január', u'február', u'március', 
                 u'április', u'május', u'június', 
                 u'július', u'augusztus', u'szeptember', 
                 u'október', u'november', u'december']
    
    y, m, d = date_groups
    
    try: 
        m = str(months_hu.index(m) + 1)
    except:
        return y, m, d
    
    return y, m, d


def get_date(desc):
    '''
        Input: description extracted from KML
        Output: Finds dates in the description, returns it in pandas datetime format
    '''
    res = dp.search(desc)
    if res is not None:
        date = standardize_date(res.groups())
    else:
        date = ''
    
    return pd.to_datetime('-'.join(date))

In [10]:
df['date'] = df['description'].apply(get_date)

In [11]:
lnp = re.compile('https?://[^ ,]*')

def get_link(desc):
    '''
        Input: description extracted from KML
        Output: finds links to external news sites
    '''
    res = lnp.search(desc)
    if res is not None:
        ln = desc[res.start():res.end()]
    else: 
        ln = ''
        
    return ln

In [12]:
df['link'] = df['description'].apply(get_link)

In [13]:
print(len(df))

df.head(15)

224


,year,location,lat,lon,description,media,date,link
0,2019,Siménfalva,46.337221,25.107726,"Kórházba került 2019. január 6-án, vasárnap eg...",,2019-01-06,
1,2019,Kőrispatak,46.417871,24.968825,2019. március 22-én 11 óra körül életveszélyes...,,2019-03-22,
2,2018,Székelyszenterzsébet,46.290531,24.935896,2018. február 6-án egy székelyszenterzsébeti f...,,2018-02-06,
3,2018,Homoródremete,46.292650,25.402600,A Homoródszentmárton községhez tartozó Homoród...,https://lh3.googleusercontent.com/Ehzdz9KPuyEb...,NaT,
4,2018,Szent Anna-tó,46.126892,25.890094,Újabb medvetámadás áldozata került kórházba Ha...,https://lh5.googleusercontent.com/j298p_0ZiD5D...,NaT,
5,2018,Tusnádfürdő,46.146075,25.856959,"Újból emberre támadt a medve Hargita megyében,...",https://lh6.googleusercontent.com/Pzc10bLlhWJM...,NaT,
6,2018,Tusnádfürdő,46.147443,25.860907,Újabb medvetámadás történt július 24-én éjjel ...,,NaT,
7,2018,Tusnádfürdő,46.143518,25.861250,2018. augusztus 5-én kora reggel a 28 éves tus...,https://lh6.googleusercontent.com/rHyfjJsaw0jk...,2018-08-05,
8,2018,Tusnádfürdő,46.143697,25.852667,2018 augusztus 5-én kora reggel a 28 éves tusn...,https://lh3.googleusercontent.com/4UDOqPpshBDm...,2018-08-05,
9,2018,Tarcsafalva,46.375859,25.130940,2018. augusztus 21-én emberre támadt a medve K...,,2018-08-21,


In [14]:
df.to_excel('data/vadkarok_hr.xlsx')

C:\Users\SPA4CLJ\AppData\Local\Continuum\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://lh3.googleusercontent.com/S82Pojbqfgo0ecnD67_WS7dbnpwe5GOJVHHy5wki68iQZKni48N2xjgf1oYP17NdHMbi2u1XnZjwXXA1dni-QEiuTTHK9Z_2mdFJz12U-gTrLVDUYT1mt_hKVA%20https://lh3.googleusercontent.com/XdleGXGQRWFO8DYvW-7e6dCw8OheLPEsn555tEC6m5IWi5kPkDu2WSczDYoNeSRZUxG4lNQ6FUE-oFpg6gf-Wl1PYolHX40BAq_2wgrb24bvaszqt2wPyMfagpI' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
